<a href="https://colab.research.google.com/github/zhaoxin1124ds/NLP-on-scientific-articles-for-information-retrieval/blob/main/NLP_Scientific_Articles_Retrieval_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install anvil uplink

In [ ]:
!pip install anvil-uplink

     |████████████████████████████████| 61kB 5.7MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-cp36-none-any.whl size=45216 sha256=aa7a208bcb52b6137ad1cfe006fbe848254def5576f2cbc919c79ca867d272af
  Stored in directory: /root/.cache/pip/wheels/a2/6e/4e/8b0ae12fb9b8a05715256952cf7609a8ab86285fab99b88c68
Successfully built ws4py


Install anvil server

In [ ]:
import anvil.server

Front-end link's connection key

In [ ]:
anvil.server.connect("QMV4ISJL6TAYCK6MPQV562S3-H6ELBKALGZIJUJN7")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


##Word2Vec

In [ ]:
import pandas as pd
import numpy as np

import gensim
from sklearn.metrics.pairwise import cosine_similarity

!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
url = 'https://drive.google.com/file/d/1OwzUFxCkdiT7DzjaEYfkXgJe2O9-AkxA/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
data_token = pd.read_csv(path)

In [ ]:
inp_list = []

for text in (data_token['sent']):
  inp_list.append(text.split())

In [ ]:
model = gensim.models.Word2Vec(inp_list, workers=8, min_count=1, window=6, sg=0, hs=1, size=300)

In [ ]:
word2vec = np.zeros((data_token.shape[0],300)) ### an array of Nx300
for i, text in enumerate(inp_list):
  word2vec[i,:] = np.mean([model[lemma] for lemma in text], axis=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
word2vec_df = pd.DataFrame(word2vec)
word2vec_df = pd.concat([data_token[['TOPIC', 'TITLE', 'ABSTRACT']], word2vec_df], axis=1)

#information retrieval

In [ ]:
def get_embedding_w2v1(token): ### to vectorize the query using trained model, query is a string

  embedding = [] ### a list to store the vector of query

  if len(token)<1: ### if query length smaller than 1 than output a vector of 0s
    return np.zeros(300)
  else:
    for tok in (token): ### for each word in the query, if it is in the vocabulary of the model, transfer it to vector
      if tok in model.wv.vocab:
        embedding.append(model[tok]) 
      else: ### if it is not in the vocabulary, give it a random value.
        embedding.append(np.random.rand(300))

  return np.mean(embedding, axis=0) ### mean value of all the words to get the vector of the query, this is same with 
                                    ### what we did when generating vectors for data base.

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'
doc_vec = word2vec_df.drop(columns=['TOPIC','TITLE','ABSTRACT'])

In [ ]:
def info_retrv(inp_string): ### information retrieval, input must be a list of string ['string1', 'string2', ..., 'stringn']

  query_vec = get_embedding_w2v1(inp_string.split())

  doc = word2vec_df[['TOPIC','TITLE','ABSTRACT']]
  doc['similarity'] = 0

  for i in range(word2vec_df.shape[0]):
  
    doc.loc[i, 'similarity'] = cosine_similarity(np.array(doc_vec.loc[i,:]).reshape(1,-1), np.array(query_vec).reshape(1,-1)).item()

  doc.sort_values(by='similarity', ascending=False, inplace=True)
  doc = doc.iloc[:5].reset_index(drop=True)

  return doc


In [ ]:
@anvil.server.callable
def top1_return(inp_string):
  doc = info_retrv(inp_string)
  out = ['Top 1 article;', 'Topic: '+ str(doc.loc[0,'TOPIC']) + ';', 'Title: '+ str(doc.loc[0,'TITLE']) + ';', 'Similarity: '+ str(doc.loc[0,'similarity'])]
  return out

@anvil.server.callable
def top2_return(inp_string):
  doc = info_retrv(inp_string)
  out = ['Top 2 article;', 'Topic: '+ str(doc.loc[1,'TOPIC']) + ';', 'Title: '+ str(doc.loc[1,'TITLE']) + ';', 'Similarity: '+ str(doc.loc[1,'similarity'])]
  return out

@anvil.server.callable
def top3_return(inp_string):
  doc = info_retrv(inp_string)
  out = ['Top 3 article;', 'Topic: '+ str(doc.loc[2,'TOPIC']) + ';', 'Title: '+ str(doc.loc[2,'TITLE']) + ';', 'Similarity: '+ str(doc.loc[2,'similarity'])]
  return out

In [ ]:
### Run this cell whenever need to connect to front-end app
### This cell will keep running forever when connecting 
### stop this cell anytime after finishing app demo, then you will also lose the connection
### APP link: https://scientific-articles-search.anvil.app/
anvil.server.wait_forever()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-pa

KeyboardInterrupt: ignored

### Test run:

In [ ]:
top1_return('polymer')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


['Top 1 article;',
 'Topic: Physics;',
 'Title: click based porous cationic polymers for enhanced carbon dioxide capture;',
 'Similarity: 0.6564209560003762']

In [ ]:
top2_return('polymer')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


['Top 2 article;',
 'Topic: Physics;',
 'Title: rigidity induced scale invariance in polymer ejection from capsid;',
 'Similarity: 0.6388111445653023']

In [ ]:
top3_return('polymer')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


['Top 3 article;',
 'Topic: Physics;',
 'Title: liquid crystal induced elasto capillary suppression of crack formation in thin colloidal films;',
 'Similarity: 0.6282966400359479']

In [ ]:
word2vec_df.head()

,TOPIC,TITLE,ABSTRACT,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,Computer Science,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,-0.448448,0.035251,-0.089019,0.007096,-0.339412,-0.024948,-0.188786,-0.060108,-0.206685,-0.003548,0.042048,-0.061465,0.129220,0.041858,-0.023266,0.126139,0.029197,0.028929,-0.262629,-0.001307,0.058143,-0.014159,-0.001000,0.065697,-0.035884,0.032026,-0.099922,0.099893,-0.104603,0.163505,-0.168879,0.138395,0.054678,0.066432,0.150720,-0.021036,-0.075519,...,0.261588,-0.046505,0.215926,-0.032483,-0.053991,-0.188811,0.127584,-0.174169,-0.157416,0.006944,0.030822,-0.120577,0.028017,-0.282736,0.009962,-0.032324,-0.006661,0.075290,-0.257030,0.049402,-0.077897,-0.057387,-0.114512,0.293980,0.042755,0.084344,-0.048697,-0.207761,0.254927,-0.056065,-0.059280,0.166122,0.485970,0.027195,0.251415,-0.109105,0.137955,0.066458,0.326031,0.103056
1,Computer Science,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,-0.477949,0.252150,0.104502,-0.599172,0.055941,0.134536,-0.124163,-0.350260,-0.414809,0.113465,-0.059642,-0.054297,-0.192648,-0.197537,0.074049,0.389749,0.016371,0.154554,-0.131592,-0.010085,0.206507,-0.162666,-0.043451,0.032030,-0.003747,-0.218698,0.096398,0.017538,-0.042595,0.370338,-0.226409,0.133652,0.245845,0.562432,0.170427,0.089181,-0.202120,...,0.523967,0.079987,-0.021293,0.236075,0.118590,-0.132271,-0.181804,-0.562538,0.093805,0.144481,0.110627,0.192142,-0.176963,-0.387312,0.144895,0.223393,-0.222542,0.439582,-0.129087,-0.315339,-0.279286,0.007606,-0.073501,0.221205,0.255396,-0.155197,0.051089,-0.133921,0.425791,-0.091376,0.433311,0.557436,0.419998,0.121057,0.346125,-0.328763,0.115979,0.105007,0.099238,0.125111
2,Mathematics,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,-0.307183,0.117733,0.533033,0.103052,0.168331,-0.137077,0.393069,-0.494980,-0.066008,-0.070225,-0.334492,0.120257,-0.246630,-0.585483,0.217331,-0.090337,0.577048,0.261208,-0.007052,0.285776,0.253724,-0.113803,0.276311,0.122123,-0.118819,-0.549138,0.246334,0.163415,-0.389296,-0.195491,0.359406,-0.044462,-0.082277,0.170363,0.084460,-0.415979,-0.058475,...,0.254627,0.233074,-0.160440,0.095125,0.197509,-0.034298,0.600327,0.076186,-0.227420,0.013842,0.219161,-0.323361,0.075231,-0.003047,0.038519,-0.233750,0.284915,-0.023618,0.156181,-0.134106,0.261896,0.207064,-0.050655,0.346918,-0.067438,-0.319247,0.059197,-0.327312,0.144327,0.060545,0.443775,0.334781,0.462823,0.097957,0.415902,-0.346801,0.417170,-0.120023,0.098406,-0.062370
3,Mathematics,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,-0.074151,0.269585,0.298697,-0.105616,-0.092386,-0.172397,0.229857,-0.545244,-0.006653,-0.092277,0.017801,-0.100552,-0.150662,-0.660694,0.012102,0.249796,0.355041,0.259081,0.214408,0.331038,0.109056,0.092551,-0.197644,0.050059,-0.045057,-0.191982,0.158939,0.069285,0.239220,0.118832,0.379045,-0.129048,0.116900,0.290216,-0.042677,-0.477637,0.124183,...,0.121803,0.004377,0.274829,0.320731,-0.054828,0.074731,0.323229,-0.069059,-0.386298,0.190576,0.130375,-0.524699,0.080479,-0.061552,-0.101024,-0.391919,0.163713,-0.003976,-0.051148,0.109306,0.219297,0.172610,0.023374,0.301593,-0.208695,-0.436762,-0.123417,-0.206296,0.250802,0.032151,0.506820,0.354676,0.199497,-0.146292,0.269955,0.251878,0.574969,0.020415,0.311737,-0.250298
4,Mathematics,On maximizing the fundamental frequency of the...,Let $\Omega \subset \mathbb{R}^n$ be a bound...,0.095667,0.232364,0.329334,-0.140332,0.201738,-0.270306,0.154323,-0.291960,-0.060248,-0.152751,-0.113698,-0.085282,-0.128719,-0.398224,0.160521,-0.047843,0.953402,0.125512,0.085510,0.212910,-0.165107,0.021546,-0.024883,0.192614,-0.289331,-